# Project in AI in Industry: Predicting Road Accident Risk

### Imports

In [9]:
%load_ext autoreload
%autoreload 2

from util.data import load_data, clean_data, split_data, train_val_test_split

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Important constants

In [10]:
TRAIN_DATA = "data/train.csv"
TEST_DATA  = "data/test.csv"
TARGET_COL = "accident_risk"

### Checking out data

In [ ]:
# Fetch train data
data = load_data(TRAIN_DATA)

data_cleaned = clean_data(data)

X, Y = split_data(data_cleaned, TARGET_COL)
X_train, X_val, X_test, Y_train, Y_val, Y_test = train_val_test_split(X, Y)

print(f'''
Sanity checks:

Data: Rows: {len(data)}, Columns: {len(data.columns)},
Sum train, val, test: {len(X_train)+len(X_val)+len(X_test)}

X train length: {len(X_train)}, Columns: {len(X_train.columns)}
X test length: {len(X_test)}, Columns: {len(X_test.columns)}

X val length: {len(X_val)}, Columns: {len(X_val.columns)}
X test length: {len(X_test)}, Columns: {len(X_test.columns)}
''')


Sanity checks

Data: Rows: 517754, Columns: 14,
Sum train, val, test: 517754

X train length: 414202, Columns: 12
X test length: 51776, Columns: 12

X val length: 51776, Columns: 12
X test length: 51776, Columns: 12



In [21]:
data.head(5)

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30
3,3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21
4,4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56


In [22]:
X_train.head(5)

,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents
393568,rural,2,0.86,35,dim,rainy,True,False,morning,False,True,1
259815,rural,3,0.08,70,night,clear,False,True,morning,True,True,1
72423,rural,4,0.33,35,night,clear,False,True,morning,True,True,2
342049,rural,3,0.10,45,night,foggy,False,True,afternoon,False,False,0
47829,rural,3,0.66,45,night,foggy,True,False,afternoon,False,False,0


In [24]:
Y_train.head(5)

393568    0.29
259815    0.37
72423     0.24
342049    0.38
47829     0.49
Name: accident_risk, dtype: float64